In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append('/coc/scratch/anarayanan68/mint/')

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers, regularizers

from mint.core import fact_model, base_model_util, base_models, primitive_models, model_builder
from mint.utils import inputs_util, config_util

In [4]:
config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__embed_cond_vary_num_prims.config'
enc_config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__embed_cond_vary_num_prims-enc_config.yml'

In [5]:
expt_root = '/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__embed_cond_vary_num_prims/5prims'

checkpoint_dir = os.path.join(expt_root, 'checkpoints/')

In [6]:
configs = config_util.get_configs_from_pipeline_file(config_path)
model_config = configs['model']
eval_config = configs['eval_config']
eval_dataset_config = configs['eval_dataset']

In [7]:
enc_config_yaml = config_util.read_yaml_config(enc_config_path)
num_primitives = 5

In [8]:
# Model build & restore

model = model_builder.build(model_config, is_training=False,
    num_primitives=num_primitives, encoder_config_yaml=enc_config_yaml, dataset_config=eval_dataset_config)

checkpoint_manager=tf.train.CheckpointManager(
    tf.train.Checkpoint(model=model),
    directory=checkpoint_dir,
    max_to_keep=None)

checkpoint_manager.restore_or_initialize()

'/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__embed_cond_vary_num_prims/5prims/checkpoints/ckpt-9999'

In [9]:
all_primitives = model.blend_vec_to_seq(np.eye(num_primitives))
all_primitives

<tf.Tensor: shape=(5, 120, 147), dtype=float32, numpy=
array([[[ 1.23601416e-02,  8.37948732e-03,  1.48097500e-02, ...,
          1.47481775e-02,  3.27057147e-04, -8.02447787e-04],
        [ 1.23652630e-02,  9.14637744e-03,  1.48073277e-02, ...,
          1.46891316e-02,  3.00287065e-04, -8.53989332e-04],
        [ 1.12359747e-02,  9.33888089e-03,  1.48040680e-02, ...,
          1.47488862e-02,  3.26976791e-04,  9.28210467e-03],
        ...,
        [ 1.42292064e-02,  9.62257851e-03,  1.47328898e-02, ...,
          1.45465108e-02,  3.62224790e-04,  1.01557532e-02],
        [ 1.12612415e-02,  9.00646113e-03,  1.48240821e-02, ...,
          1.47614479e-02,  2.92878540e-04,  1.04221907e-02],
        [ 1.15507375e-02,  9.50073265e-03,  1.47932386e-02, ...,
          1.46513898e-02,  3.26723210e-04,  1.11128611e-05]],

       [[ 7.52923265e-03,  7.52923265e-03,  7.52923265e-03, ...,
          7.52923265e-03,  7.52923265e-03,  7.52923265e-03],
        [ 7.52923265e-03,  7.52923265e-03,  7.52

In [10]:
save_dir = os.path.join(expt_root, 'primitives')
os.makedirs(save_dir, exist_ok=True)
for i in range(num_primitives):
    np.save(os.path.join(save_dir, f"PRIMITIVE--{i+1}_of_{num_primitives}.npy"), all_primitives[i])